# Widget Experiments

This notebook is primarily being used to experiment with how to use the `pymbe` widgets.

It should be fairly "clean" until the `Scratch Pad` section.

In the future, this notebook will be cleaned up and renamed "Tutorial".

In [ ]:
import ipywidgets as ipyw
import pymbe.api as pm

In [ ]:
client = pm.SysML2ClientWidget(host="http://sysml2-sst.intercax.com")
details = pm.ElementDetails()
ipyw.VBox([client, details])

In [ ]:
client.project_selector.value = "03e3cdfe-8d90-4fc3-a414-9a188203d349"
client._download_elements()
details.update(client.elements_by_id)

In [ ]:
# Create a Labelled Property Graph
lpg = pm.SysML2LPGWidget()
lpg

In [ ]:
# Load data from client into the LPG
lpg.update(client.elements_by_id)

# Scratch Pad
> **WARNING**: Anything below this point is just for experimentation purposes

In [ ]:
display(JSON(elements_by_id))

In [ ]:
JSON(client.elements_by_id[tree.selected_nodes[0]._identifier])

In [ ]:
tree.selected_nodes

In [ ]:
import ipywidgets as ipyw
from IPython.display import JSON, display

values = []


def log_node_clicks(*args, **kwargs):
    node = tree.selected_nodes[0]
    data = client.elements_by_id[tree.selected_nodes[0]._identifier]
    data.pop("@context", None)
    with json_output:
        json_output.clear_output()
        display(JSON(data))


tree.observe(log_node_clicks, "selected_nodes")

# create output widget
json_output = ipyw.Output()
full_display = ipyw.GridBox(
    children=[tree, json_output],
    layout=ipyw.Layout(grid_template_columns="repeat(2, 500px)", height="450px"),
)

full_display

In [ ]:
import traitlets as trt
from ipywidgets import link, HBox, VBox, IntSlider, Text, Label
from ipytree import Tree, Node

self = client
# tree = Tree(stripes=True)

class Element(Node):
    _identifier = trt.Unicode()
    _owner = trt.Unicode(Element, allow_none=True)
    _type = trt.Unicode()



nodes = {
    element_id: Element(
        name=element["name"] or f"""«{element["@type"]}: {element_id}»""",
        _identifier=element["identifier"],
        _type=element["@type"],
        _owner=(element["owner"] or {}).get("@id", None),
    )
    for element_id, element in self.elements_by_id.items()
    if element.get("name", element.get("qualifiedName", None))
    or (element["owner"] or {}).get("@id", None)
}
for node in nodes.values():
    owner = node._owner
    if owner is None:
        tree.add_node(node)
    elif owner in nodes:
        nodes[owner].add_node(node)
    else:
        print(f"Not including {node} because its owner "
              f"{self.elements_by_id[owner]['qualifiedName']} does not have a name!"
        )

tree

In [ ]:
for node in tree.nodes:
    tree.remove_node(node)

In [ ]:
# Create a widget to see the qualified name of the elements nodes selected
import ipywidgets as ipyw

out = ipyw.Output(layout={'border': '1px solid black'})

def update_node_selections(*_):
    out.outputs = []
    with out:
        [
            print(element["qualifiedName"])
            for element in lpg.selected_nodes_by_type
        ]
            
lpg.node_type_selector.observe(update_node_selections, "value")

out

In [ ]:
from pymbe.widget.diagram import Diagram

diagram = Diagram()

def update_diagram(diagram, client):
    graph = client.lpg
    sub_graph = diagram.get_sub_graph(
        edge_types=
    )

diagram.graph = 

In [ ]:
def process_edge(source, target, edge_type, data=None):
    if edge_type in ("FeatureTyping", "FeatureMembership"):
        source, target = target, source
    return [source, target, data]

graph = nx.DiGraph()
graph.add_edges_from([
    process_edge(source, target, edge_type, data)
    for (source, target, edge_type), data in dict(client.lpg.graph.edges).items()
    if edge_type in ("Superclassing", "FeatureTyping", "FeatureMembership")
])
graph

In [ ]:
diagram.graph = graph

In [ ]:
diagram = client.lpg.make_diagram(
    graph=client.lpg.subgraph(edge_types=("Superclassing", "FeatureTyping^-1", "FeatureMembership^-1"))
)
elk_app, *_ = diagram.children
diagram

In [ ]:
# Select the root node in the diagram...
# ... or you can manually select one yourself
elk_app.selected = "5260380b-6fda-43cc-993f-5df58868edbb",

In [ ]:
first_element_selected, *_ = elk_app.selected
client.elements_by_id[first_element_selected]

# Parse JSON-LD into RDF

In [ ]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

result = client.rdf.graph
# result = g.parse(url, format='turtle')

G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')

In [ ]:
ax = plt.figure(figsize=(50,30)).gca();
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
nx.draw(G, with_labels=True, ax=ax)

# TODOs
1. Finish fleshing out the process in the `Kerbal Model.ipynb`
2. Modify the subgraph generator so it can take the value from the `Type Selector` directly
3. Improve the ipyelk diagram widget (may need to make improvements to `ipyelk`)
   * Add arrows
   * Add compartments
   * Fix layout
   * Add widget to see node details
4. Finalize the RDF formulation